In [ ]:
%pip install sqlglot

In [ ]:
import sqlglot
from sqlglot import parse_one, exp
from sqlglot.lineage import lineage
from sqlglot.schema import MappingSchema
from sqlglot.dialects import Snowflake

In [ ]:
sql = """


with a as (
    select 
       original as col1, 
       col2
    from 
       t1
)
select
    col2 as col1
from
  a
where col1 = 3902
"""
node = lineage(
    "col1",
    sql,
    dialect=Snowflake
)

source_column_lineage = []
start = node.name

source_column_lineage.append(start)

next_downstream = node.downstream[0]
while True:
    source_column_lineage.append(next_downstream.name)
    if len(next_downstream.downstream) == 0:
        break
    next_downstream = next_downstream.downstream[0]

source_column_lineage

In [ ]:
a = node.downstream[0].downstream[0]

In [ ]:
parsed = parse_one(sql)

for j in parsed.find_all(exp.Where):
    print(f"where: {j}")
    print(repr(j))

In [ ]:
type(node)

In [ ]:
j.find_all(exp.Predicate)

In [ ]:
j.flatten()

In [ ]:
for i in j.flatten():
  print(type(i))
  print(i)
    
for table in j.find_all(exp.Column):
  print(f"table: {table.name}")

    # for cond in j.find_all(exp.Predicate):
    #   print(repr(cond))
    #   print(f"cond: {type(cond)}")
    # print(f"join on: {j.on}")

In [ ]:
for e in j.find_all(exp.Column):
    print("hello")
    print(e)

In [ ]:
j